In [ ]:
import numpy as np
import pandas as pd
import joblib
import json
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

TOP_K = 10
RANDOM_STATE = 42
MODEL_PATH = "reduced_model.h5"
SCALER_PATH = "reduced_scaler.pkl"
FEATURES_JSON = "selected_features.json"

data = load_breast_cancer()
X = data.data
y = data.target
feature_names_all = list(data.feature_names)

rf = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1)
rf.fit(X, y)
importances = rf.feature_importances_
feat_imp_df = pd.DataFrame({"feature": feature_names_all, "importance": importances})
feat_imp_df = feat_imp_df.sort_values("importance", ascending=False).reset_index(drop=True)

selected_features = feat_imp_df['feature'].tolist()[:TOP_K]
selected_indices = [feature_names_all.index(f) for f in selected_features]

print("Selected features (top {}):".format(TOP_K))
print(selected_features)

X_reduced = X[:, selected_indices]
X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def build_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_model(X_train_scaled.shape[1])
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=50,
    batch_size=16,
    verbose=1
)

loss, acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test accuracy on reduced features: {acc:.4f}")

model.save(MODEL_PATH)
joblib.dump(scaler, SCALER_PATH)
with open(FEATURES_JSON, "w") as f:
    json.dump(selected_features, f)

print("Saved:", MODEL_PATH, SCALER_PATH, FEATURES_JSON)

Selected features (top 10):
['worst perimeter', 'worst area', 'worst concave points', 'mean concave points', 'worst radius', 'mean area', 'mean perimeter', 'mean concavity', 'mean radius', 'worst concavity']
Epoch 1/50
29/29 [==============================] - 2s 16ms/step - loss: 0.5524 - accuracy: 0.7429 - val_loss: 0.3603 - val_accuracy: 0.9211
Epoch 2/50
29/29 [==============================] - 0s 7ms/step - loss: 0.2553 - accuracy: 0.9429 - val_loss: 0.1968 - val_accuracy: 0.9211
Epoch 3/50
29/29 [==============================] - 0s 5ms/step - loss: 0.1625 - accuracy: 0.9407 - val_loss: 0.1612 - val_accuracy: 0.9211
Epoch 4/50
29/29 [==============================] - 0s 6ms/step - loss: 0.1530 - accuracy: 0.9473 - val_loss: 0.1516 - val_accuracy: 0.9123
Epoch 5/50
29/29 [==============================] - 0s 7ms/step - loss: 0.1404 - accuracy: 0.9429 - val_loss: 0.1479 - val_accuracy: 0.9123
Epoch 6/50
29/29 [==============================] - 0s 4ms/step - loss: 0.1363 - accuracy: 

c:\Users\jishn\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
